$\newcommand{\if}[1]{{\textbf{if }{#1}\textbf{ then:}}}$
$\newcommand{\else}{{\textbf{else:}}}$

# Shor's Algorithm

Shor's algorithm accepts a (large) integer $N$ and finds $p$, $q$ such that $N=p\cdot q$ and both $p$, $q$ are prime

---

**Algorithm Shor**

**Input:** The integer to factor, $N$

**Output:** $p,q$ such that $N = p \cdot q$

**begin**

1. Choose a random integer $1 < a < N$
2. $b := \gcd(a,N)$
3. $\if{b \neq 1}$
4. &emsp;$b$ factors $N$; return $p=b$ and $q=N/b$
5. $\else$
6. &emsp;Use quantum subroutine to find the order $r$ of $a$
7. &emsp;$\if{r\text{ is odd}}$
8. &emsp;&emsp;go to line 1
9. &emsp;$g := \gcd(a^{r/2}\pm 1, N)$
10. &emsp;$\if{g>1}$
11. &emsp;&emsp;return $p=g$, $q=N/g$
12. &emsp;$\else$
13. &emsp;&emsp;go to line 1

**end**

---


## Quantum Period Finding Algorithm

---

**Algorithm QCalculatePeriod**

**Input:** Coprime integers $N$ and $1 \lt a \lt N$

**Output:** The order of $a$ (i.e., $\min(r>0)$ such that $a^r = 1\pmod{N}$)

**begin**

1. $m := \lceil \lg N \rceil$
2. $n := 2m$
3. Prepare two registers, one of size $n$ and the other of size $m$: $\ket{\Psi} = \ket{0}^{\otimes n}\ket{\psi}^{\otimes m}$
4. Apply a Hadamard transform to the first register in $\ket{\Psi}$
5. Apply a unitary operator $U$ to the second register, where $U\ket{\psi} = e^{i2 \pi \theta}\ket{\psi}$
6. Compute the inverse Quantum Fourier Transform on the first register
7. Measure the first register to find the most probable $k$ such that $k=qN/r$ ($q$ is a random integer $\in \left[ 0, r \right)$
8. Compute the continued fraction approximation of $N/k$. It is likely that this approximation will be an integer multiple of the period $r$.

**end**

Note: $n$ is proportional to the accuracy of the algorithm; $n$ is arbitrary and does not necessarily have to be defined as $2m$ in Step 2.

---

## Continued Fractions Algorithm

The Continued Fractions Algorithm finds the continued fraction expansion for any irrational number to as many partial denominators as desired

The following system of equations is a way of describing the continued fractions algorithm

$$
\begin{align*}
   x &= a_0 + b_0 \\
   \frac{1}{b_0} &= a_1' = a_1 + b_1 \\
   &\vdots\\
   \frac{1}{b_k} &= a_{k+1}' = a_{k+1} + b_{k+1}
\end{align*}
$$

Source: [Cornell](https://pi.math.cornell.edu/~gautam/ContinuedFractions.pdf)

---

**Algorithm ContinuedFraction(x)**
 
**Input:** A number $x \in \mathbb{R}$

**Output:** Continued fraction representation of $x$: $\left[ a_0, a_1, a_2, ... \right]$ such that


**begin**

1. $a_m := \lfloor$ (i.e., let $a_m$ be the integer part of $x_m$)
2. $b_m := x_m - a_m$
3. $\if{b_m \neq 0}$
4. &emsp; Set $x_{m+1} = 1/b_m$ and go back to step 1 to compute $a_{m+1}$
5. $\else$
6. &emsp; return

**end**

Note: as some sequences can grow very large, we define a positive $\epsilon \ll 1$ so that instead of directly testing $b_m = 0$, we instead test $|b_m| < \epsilon$. This will allow us to check if $b_m$ is approximately zero.

---

In [1]:
import numpy as np

def ContinuedFraction(x, eps=1e-8, retlist=False):
    """
    Return approximate fraction representation of a value.

    Parameters
    ----------
        eps : int, optional
            The threshold for approximating bm to 0. Default: 1e-8.
        retlist : bool, optional
            if True, also returns the list of coefficients for the representation.

    Returns
    -------
        numr: int
            The numerator of the approximated fraction.
        denr: int
            Only returned is `retlist` is False
            
            The denominator of the approximated fraction.
        coeffs: ndarray
            Only returned if `retlist` is True

            Coefficient representation of the approximated fraction.
    """
    
    def BuildContinuedFractionRepresentation(xm, a: np.ndarray):
        # Compute the integer part of xm
        am = int(xm)                      
        
        # Add integer part of xm to the results list (denominator of continued fractions)                                
        a = np.append(a, am)
        
        # Compute denominator of x_{m+1}                                  
        bm = xm - am                                                      
        
        # Terminate if bm ~ 0 
        if np.abs(bm) < eps:
            return a
        else:
            # Compute a_{m+1}
            return BuildContinuedFractionRepresentation(1/bm, a)

    # Get coefficients of approximated fraction 
    coeffs = BuildContinuedFractionRepresentation(x, np.array([], dtype=int))

    # https://math.stackexchange.com/questions/3084970/how-to-convert-continued-fractions-into-normal-fractions
    H = np.empty(len(coeffs) + 2)
    K = np.empty_like(H)
    H[0], H[1] = 0, 1
    K[0], K[1] = 1, 0
    for n in range(2, len(H)):
        H[n] = coeffs[n-2] * H[n-1] + H[n-2]
        K[n] = coeffs[n-2] * K[n-1] + K[n-2]

    numr = H[-1]
    denr = K[-1]

    if retlist:
        return numr, denr, coeffs
    
    return numr, denr

# tests

assert (ContinuedFraction(2.875) == (23, 8))
assert (ContinuedFraction(2.875, retlist=True)[2] == [2,1,7]).all()

In [112]:
from qiskit import ClassicalRegister, QuantumCircuit, QuantumRegister
import math
from qiskit.circuit.library import QFT

# inputs
# ----------------------
a, N = 7, 25
eps = 1e-1
# ----------------------

In [125]:
L = 1 + 2 * math.ceil( math.log(N, 2) )
m = L + math.ceil( math.log(2 + 1/2/eps, 2) )
n = m
print(f'n,m = {n,m}')

# First quantum register for Phase Estimation
# Classical register for storing measurements
qr1 = QuantumRegister(n, name='0 register')
qr2 = QuantumRegister(m, name='ψ register')
crr = ClassicalRegister(n, name='results')

# Create circuit
shor = QuantumCircuit(qr1, qr2, crr)


#####

# Apply Hadamard gate to first register
shor.h(qr1)

# this is where we apply the Unitary operator to the second register
# --------------------------------------------------------

shor.x(-1)
def c_amodN(a, power, num_qubits):
    U = QuantumCircuit(num_qubits//2)
    for _ in range(power):
        if a%2 != 0:
            for q in range(num_qubits//2):
                U.x(q)
    U = U.to_gate()
    U.name = f'{a}^{power} mod N'
    return U.control()

for q in range(n):
    cir = c_amodN(a, 2**q, n)
    shor.append(cir, [q] + [i+n for i in range(n//2)])
    
# --------------------------------------------------------

#Apply inverse QFT to the first Register
shor.append(QFT(n, inverse=True), qr1)

#Measure the first register 
shor.measure(qr1, crr)

#Execute the ciruit 
shor.draw(fold=-1)

n,m = (14, 14)


┌───┐                                                                                                                                                                                                                         ┌────────┐┌─┐                                       
 0 register_0: ┤ H ├──────■──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤0       ├┤M├───────────────────────────────────────
               ├───┤      │                                                                                                                                                                                                                  │        │└╥┘┌─┐                                    
 0 register_1: ┤ H ├──────┼─────────────■────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤1       ├─╫─┤M├────────────────────────────────────
               ├───┤      │             │                                                                                                                                                                                                    │        │ ║ └╥┘┌─┐                                 
 0 register_2: ┤ H ├──────┼─────────────┼─────────────■──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤2       ├─╫──╫─┤M├─────────────────────────────────
               ├───┤      │             │             │                                                                                                                                                                                      │        │ ║  ║ └╥┘┌─┐                              
 0 register_3: ┤ H ├──────┼─────────────┼─────────────┼─────────────■────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤3       ├─╫──╫──╫─┤M├──────────────────────────────
               ├───┤      │             │             │             │                                                                                                                                                                        │        │ ║  ║  ║ └╥┘┌─┐                           
 0 register_4: ┤ H ├──────┼─────────────┼─────────────┼─────────────┼──────────────■─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤4       ├─╫──╫──╫──╫─┤M├───────────────────────────
               ├───┤      │             │             │             │              │                                                                                                                                                         │        │ ║  ║  ║  ║ └╥┘┌─┐                        
 0 register_5: ┤ H ├──────┼─────────────┼─────────────┼─────────────┼──────────────┼──────────────■──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤5       ├─╫──╫──╫──╫──╫─┤M├────────────────────────
               ├───┤      │             │             │             │              │              │                                                                                                                                          │        │ ║  ║  ║  ║  ║ └╥┘┌─┐                     
 0 register_6: ┤ H ├──────┼─────────────┼─────────────┼─────────────┼──────────────┼──────────────┼──────────────■───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤6      

In [ ]:
from qiskit import transpile
from qiskit.circuit.library import RealAmplitudes
from qiskit.quantum_info import SparsePauliOp
from qiskit_aer import Aer, AerSimulator
from qiskit_aer.primitives import SamplerV2

simulator = AerSimulator()
circuit = transpile(shor, simulator)

result = simulator.run(circuit, shots=1, memory=True).result()
memory = result.get_memory()
phase = int(memory[0],2)/(2**n)
memory, phase, ContinuedFraction(phase)